In [1]:
import pandas as pd
import numpy as np

import source.hyperplots as hyp
import source.utility as ut
import source.report as rp
import source.transf_univ as df_p

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, log_loss, accuracy_score

from sklearn.linear_model import Ridge, LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

pd.set_option('max_columns', 300)

In [2]:
df = pd.read_csv('data/processed_men/training_data.csv')

df.head()

,Season,DayNum,Team1,Team2,target,target_points,T1_Loc,T2_Loc,ID,T1_Score,T1_FGM,T1_FGA,T1_FGM3,T1_FGA3,T1_FTM,T1_FTA,T1_OR,T1_DR,T1_Ast,T1_TO,T1_Stl,T1_Blk,T1_PF,T1_FGM2,T1_FGA2,T1_Tot_Reb,T1_FGM_no_ast,T1_possessions,T1_off_rating,T1_scoring_opp,T1_TO_perposs,T1_def_rating,T1_impact,T1_Def_effort,T1_Reb_opp,T1_Score_diff,T1_FGM_diff,T1_FGA_diff,T1_FGM3_diff,T1_FGA3_diff,T1_FTM_diff,T1_FTA_diff,T1_OR_diff,T1_DR_diff,T1_Ast_diff,T1_TO_diff,T1_Stl_diff,T1_Blk_diff,T1_PF_diff,T1_FGM2_diff,T1_FGA2_diff,T1_Tot_Reb_diff,T1_FGM_no_ast_diff,T1_Def_effort_diff,T1_Reb_opp_diff,T1_possessions_diff,T1_off_rating_diff,T1_def_rating_diff,T1_scoring_opp_diff,T1_TO_perposs_diff,T1_impact_diff,T1_N_wins,T1_FGM_perc,T1_FGM2_perc,T1_FGM3_perc,T1_FT_perc,T1_FGM_no_ast_perc,T1_L2W_Score,T1_L2W_FGM,T1_L2W_FGA,T1_L2W_FGM3,T1_L2W_FGA3,T1_L2W_FTM,T1_L2W_FTA,T1_L2W_OR,T1_L2W_DR,T1_L2W_Ast,T1_L2W_TO,T1_L2W_Stl,T1_L2W_Blk,T1_L2W_PF,T1_L2W_FGM2,T1_L2W_FGA2,T1_L2W_Tot_Reb,T1_L2W_FGM_no_ast,T1_L2W_possessions,T1_L2W_off_rating,T1_L2W_scoring_opp,T1_L2W_TO_perposs,T1_L2W_def_rating,T1_L2W_impact,T1_L2W_Def_effort,T1_L2W_Reb_opp,T1_L2W_Score_diff,T1_L2W_FGM_diff,T1_L2W_FGA_diff,T1_L2W_FGM3_diff,T1_L2W_FGA3_diff,T1_L2W_FTM_diff,T1_L2W_FTA_diff,T1_L2W_OR_diff,T1_L2W_DR_diff,T1_L2W_Ast_diff,T1_L2W_TO_diff,T1_L2W_Stl_diff,T1_L2W_Blk_diff,T1_L2W_PF_diff,T1_L2W_FGM2_diff,T1_L2W_FGA2_diff,T1_L2W_Tot_Reb_diff,T1_L2W_FGM_no_ast_diff,T1_L2W_Def_effort_diff,T1_L2W_Reb_opp_diff,T1_L2W_possessions_diff,T1_L2W_off_rating_diff,T1_L2W_def_rating_diff,T1_L2W_scoring_opp_diff,T1_L2W_TO_perposs_diff,T1_L2W_impact_diff,T1_L2W_N_wins,T1_L2W_FGM_perc,T1_L2W_FGM2_perc,T1_L2W_FGM3_perc,T1_L2W_FT_perc,T1_L2W_FGM_no_ast_perc,T1_Seed,T2_Score,T2_FGM,T2_FGA,T2_FGM3,T2_FGA3,T2_FTM,T2_FTA,T2_OR,T2_DR,T2_Ast,T2_TO,T2_Stl,T2_Blk,T2_PF,T2_FGM2,T2_FGA2,T2_Tot_Reb,T2_FGM_no_ast,T2_possessions,T2_off_rating,T2_scoring_opp,T2_TO_perposs,T2_def_rating,T2_impact,...,T2_L2W_FGM_diff,T2_L2W_FGA_diff,T2_L2W_FGM3_diff,T2_L2W_FGA3_diff,T2_L2W_FTM_diff,T2_L2W_FTA_diff,T2_L2W_OR_diff,T2_L2W_DR_diff,T2_L2W_Ast_diff,T2_L2W_TO_diff,T2_L2W_Stl_diff,T2_L2W_Blk_diff,T2_L2W_PF_diff,T2_L2W_FGM2_diff,T2_L2W_FGA2_diff,T2_L2W_Tot_Reb_diff,T2_L2W_FGM_no_ast_diff,T2_L2W_Def_effort_diff,T2_L2W_Reb_opp_diff,T2_L2W_possessions_diff,T2_L2W_off_rating_diff,T2_L2W_def_rating_diff,T2_L2W_scoring_opp_diff,T2_L2W_TO_perposs_diff,T2_L2W_impact_diff,T2_L2W_N_wins,T2_L2W_FGM_perc,T2_L2W_FGM2_perc,T2_L2W_FGM3_perc,T2_L2W_FT_perc,T2_L2W_FGM_no_ast_perc,T2_Seed,delta_Loc,delta_Score,delta_FGM,delta_FGA,delta_FGM3,delta_FGA3,delta_FTM,delta_FTA,delta_OR,delta_DR,delta_Ast,delta_TO,delta_Stl,delta_Blk,delta_PF,delta_FGM2,delta_FGA2,delta_Tot_Reb,delta_FGM_no_ast,delta_possessions,delta_off_rating,delta_scoring_opp,delta_TO_perposs,delta_def_rating,delta_impact,delta_Def_effort,delta_Reb_opp,delta_Score_diff,delta_FGM_diff,delta_FGA_diff,delta_FGM3_diff,delta_FGA3_diff,delta_FTM_diff,delta_FTA_diff,delta_OR_diff,delta_DR_diff,delta_Ast_diff,delta_TO_diff,delta_Stl_diff,delta_Blk_diff,delta_PF_diff,delta_FGM2_diff,delta_FGA2_diff,delta_Tot_Reb_diff,delta_FGM_no_ast_diff,delta_Def_effort_diff,delta_Reb_opp_diff,delta_possessions_diff,delta_off_rating_diff,delta_def_rating_diff,delta_scoring_opp_diff,delta_TO_perposs_diff,delta_impact_diff,delta_N_wins,delta_FGM_perc,delta_FGM2_perc,delta_FGM3_perc,delta_FT_perc,delta_FGM_no_ast_perc,delta_L2W_Score,delta_L2W_FGM,delta_L2W_FGA,delta_L2W_FGM3,delta_L2W_FGA3,delta_L2W_FTM,delta_L2W_FTA,delta_L2W_OR,delta_L2W_DR,delta_L2W_Ast,delta_L2W_TO,delta_L2W_Stl,delta_L2W_Blk,delta_L2W_PF,delta_L2W_FGM2,delta_L2W_FGA2,delta_L2W_Tot_Reb,delta_L2W_FGM_no_ast,delta_L2W_possessions,delta_L2W_off_rating,delta_L2W_scoring_opp,delta_L2W_TO_perposs,delta_L2W_def_rating,delta_L2W_impact,delta_L2W_Def_effort,delta_L2W_Reb_opp,delta_L2W_Score_diff,delta_L2W_FGM_diff,delta_L2W_FGA_diff,delta_L2W_FGM3_diff,delta_L2W_FGA3_diff,delta_L2W_FTM_diff,delta_L2W_FTA_diff,delta_L2W_OR_diff,delta_L2W_DR_diff,delta_L2W_Ast

* Both on target and target_points
* General 80/20 split, Kfold and test results
* kfold on n-1 season and test on the remaining one

In [17]:
def random_split(data, model, kfolds, target, test_size=0.2, predict_proba=False):
    
    train, test = ut.make_test(data, test_size=test_size, random_state=324)
    
    y_train = train[target]
    y_test = test[target]
    
    for col in ['target', 'target_points', 'ID', 'DayNum', 'Team1', 'Team2', 'Season']:
        try:
            del train[col]
            del test[col]
        except KeyError:
            pass
    
    oof, imp_coef = ut.cv_score(train, y_train, kfolds, model, imp_coef=True, predict_proba=predict_proba)
    
    fit_model = model.fit(train, y_train)    
    
    return fit_model, oof, imp_coef, train, y_train, test, y_test

In [18]:
df_train = df[['Season', 'target', 'target_points', 'ID', 'delta_Seed', 'delta_Score']]

df_train.head()

,Season,target,target_points,ID,delta_Seed,delta_Score
0,2003,0,-8,2003_1411_1421,0.0,1.593103
1,2003,1,29,2003_1112_1436,-15.0,17.421182
2,2003,1,13,2003_1113_1272,3.0,1.448276
3,2003,1,6,2003_1141_1166,5.0,0.102403
4,2003,1,2,2003_1143_1301,-1.0,2.082759


In [11]:
pipe = Pipeline([('scl', df_p.df_scaler()), ('ridge', Ridge())])

kfolds = KFold(n_splits=5, shuffle=True, random_state=345)

fitted, oof_pred, imp_coef, train, y_train, test, y_test = random_split(df_train, pipe, kfolds, 'target_points')

In [12]:
imp_coef

,mean,std
feat,,
delta_Seed,-8.194218,0.129185
delta_Score,0.738528,0.118569


In [13]:
pipe = Pipeline([('scl', df_p.df_scaler()), ('logit', LogisticRegression(solver='lbfgs', multi_class='auto'))])

fitted, oof_pred, imp_coef, train, y_train, test, y_test = random_split(df_train, pipe, kfolds, 'target', predict_proba=True)

In [14]:
imp_coef

,mean,std
feat,,
delta_Seed,-1.161836,0.044797
delta_Score,0.128212,0.034697


In [19]:
def yearly_split(data, model, kfolds, target, predict_proba=False):
    
    fit_model = {}
    oof = {}
    imp_coef = {}
    train = {}
    test = {}
    y_train = {}
    y_test = {}    
    
    for year in data.Season.unique():
        yr = str(year)
        train[yr] = data[data.Season != year].copy()
        test[yr] = data[data.Season == year].copy()
    
        y_train[yr] = train[yr][target]
        y_test[yr] = test[yr][target]

        for col in ['target', 'target_points', 'ID', 'DayNum', 'Team1', 'Team2', 'Season']:
            try:
                del train[yr][col]
                del test[yr][col]
            except KeyError:
                pass
    
        oof[yr], imp_coef[yr] = ut.cv_score(train[yr], y_train[yr], kfolds, model, imp_coef=True, predict_proba=predict_proba)
    
        fit_model[yr] = model.fit(train[yr], y_train[yr])    
    
    return fit_model, oof, imp_coef, train, y_train, test, y_test

In [20]:
pipe = Pipeline([('scl', df_p.df_scaler()), ('ridge', Ridge())])

fitted, oof_pred, imp_coef, train, y_train, test, y_test = yearly_split(df_train, pipe, kfolds, 'target_points')

In [22]:
fitted.keys()

dict_keys(['2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018'])